# 언어 모델

단어 시퀀스에 확률을 할당하는 모델
크게 통계를 이용한 방법과 인공신경망을 이용한 방법으로 구분

ex) 두 문장간 확률 비교
    a. 기계 번역
    b. 오타 교정
    c. 음성 인식

ex) 이전 단어로부터 다음 단어 예측


하나의 단어 w, 단어 시퀀스 W (n개의 단어:w1,w2,..wn)
A. 단어 시퀀스의 확률
B. 다음 단어 등장 확률

1. 통계적 언어 모델

     1) 조건부 확률 : chain rule !!
     2) 문장에 대한 확률
        P(An adorable little boy is spreading smiles)= 
        P(An)×P(adorable|An)×P(little|An adorable)×P(boy|An adorable little)×P(is|An adorable little boy) ×P(spreading|An adorable little boy is)×P(smiles|An adorable little boy is spreading)
     3) 카운트 기반의 접근 
            - 한계 : 희소문제(Sparsity problem)





2. N-gram 언어 모델

:이전에 등장한 모든 단어를 고려하는 것이 아니라 일부 단어만 고려하는 접근 방법
  , 앞의 n-1개의 단어만 고려

BUT, 한국어에서의 언어모델은 어렵다
why? 
1. 어순이 중요하지 않다
2. 교착어이다
3. 띄어쓰기를 제대로 안한다.

3. 성능비교기준

1) 플렉서티(perplexity) : 낮을수록 언어모델의 성능의 좋다는 것을 의미함


2) 분기 계수(Brancing factor) : 특정 시점에서 평균적으로 몇개의 선택지를 가지고 고민하는지 의미


# 다양한 단어 표현 방법


1) 국소 표현(Local Representation) = 이산표현(Discrete Representation)
ex) One-hot-vector, N-gram, Count-based; Bag of Words(DTM)

2) 분산표현(Distributed Representation) = 연속표현(Continuous Representation) : 늬앙스 표현 가능
ex) Prediction based; Word2Vec(FastText), Count based;Full document(LSA), Windows(glove)
 

1. Bag of Word(BoW)

(1) 우선, 각 단어에 고유한 정수 인덱스를 부여합니다.

(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

In [1]:
##실습 코딩

from konlpy.tag import Okt
import re  
okt=Okt()  

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")  
# 정규 표현식을 통해 온점을 제거하는 정제 작업입니다.  
token=okt.morphs(token)  
# OKT 형태소 분석기를 통해 토큰화 작업을 수행한 뒤에, token에다가 넣습니다.  

word2index={}  
bow=[]  
for voca in token:  
         if voca not in word2index.keys():  
             word2index[voca]=len(word2index)  
# token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.   
             bow.insert(len(word2index)-1,1)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.  
         else:
            index=word2index.get(voca)
# 재등장하는 단어의 인덱스를 받아옵니다.
            bow[index]=bow[index]+1
# 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다. (단어의 개수를 세는 것입니다.)  
print(word2index)  

C:\Users\PC\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


2. 문서단어행렬

서로 다른 문서들의 BoW들을 결합한 표현 방법 
: DTM, Document-Term Matrix

rows = 문서1,2,3,... / cols= 나는, 어제, .. (단어들) / value = bow값
-> 한계 : 희소 표현, 단어 빈도 수 기반 접근

    
TF-IDF(Term Frequency-Inverse Document Frequency) :
    

3.TF-IDF(Term Frequency-Inverse Document Frequency)

DTM 내에 있는 각 단어에 대한 중요도 계산할 수 있는 가중치

    1) 주로 문서의 유사도를 구하는 작업,
    2) 검색 시스템에서 검색 결과의 중요도를 정하는 정하는 작업, 
    3) 문서 내에서 특정 단어의 중요도를 구하는 작업
    
식) 문서 d, 단어 t, 문서의 총 갯수 n

tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수

df(t) : 특정 단어 t가 등장한 문서의 수

idf(d,t) : df(t)에 반비례 하는 수 log(n/1+df(t)) -> 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단

최종식 : tf * idf

# 문서유사도

In [ ]:
## 문서유사도
1) 코사인 유사도
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])
print(cos_sim(doc1, doc2)) 
print(cos_sim(doc1, doc3)) 
print(cos_sim(doc2, doc3))

2)유클리드 거리
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

3)자카드 유사도(Jaccard similarity)
두 개의 비교할 문서를 각각 doc1, doc2라고 했을 때 doc1과 doc2의 문서의 유사도를 구하기 위한 자카드 유사도는 이와 같습니다.
즉, 두 문서 doc1, doc2 사이의 자카드 유사도 J(doc1,doc2)는 두 집합의 교집합 크기를 두 집합의 합집합 크기로 나눈 값으로 정의됩니다.
J(doc1,doc2)=doc1∩doc2 / doc1∪doc2


# 토픽 모델링

본문의 숨겨진 의미 구조를 발견하기 위해 사용

1) LSA ( Latent Semantic Analysis )
* SVD ( Singular Value Decomposition , 특이값 분해) : A=UΣVT
-> 차원 축소 및 잠재적인 의미 포착 기능 
-> full SVD와 Truncated SVD(t)로 두 가지로 나뉨
: 축소된 U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터
: 축소된 VT의 각 열은 잠재 의미르 표현하기 위해 수치화된 각각의 단어벡터    